# An Empirical Model of Large-Batch Training

## Summary

* 상대적으로 큰 batch를 쓰면 보다 빠르게 원하는 성능 도달, 그러나 너무 크면 효과 없음
* 도메인마다 적절한 배치 사이즈가 다르다. 
* 새로운 도메인에 적합한 배치 사이즈를 가늠할 통계치인 gradient noise scale을 제안한다.
* 이는 해당 도메인에서의 neural network의 병렬 처리 용량에 대한 고찰이라고 볼 수 있다. 
* 또한 해당 도메인에 대한 문제 복잡도(난이도)에 대한 가늠자이기도 하다. 

## 도입부

* 딥모델의 학습을 위해 많은 계산량이 요구되고 이를 잘 실현해 나가고 있다. 
* 이를 가능케 한 것이 (GPU를 사용한) 데이터 병렬처리였다. 
* 데이터 병렬성(data parallism)은 이를 다루는 알고리즘(보통 SGD)이 이를 받쳐주어야 한다. 
* 최근 연구에 따르면 경험적으로 산출한 상대적으로 큰 배치 사이즈가 sample efficiency나 일반화 성능 저하 없이 학습을 가속화(speed-up) 시킬 수 있음을 밝혔다. 
* 하지만 이는 경험적인 것이고, 다음 궁금증이 생기는 것이다. 
  * 이러한 적당한 배치 사이즈를 어떻게 시스템적으로 찾아야 하는 것일까? (절차 필요)
  * 왜 이런 값을 가지게 되는 것일까? (이론 필요)
  * 왜 도메인마다/모델마다 다른 걸까? (확장성 고민)

## 적절한 배치가 좋을 수 있다는 직관적 설명
 
* small batch는 상대적으로 noisy(deviated from true gradient)해서 많이 해매다가 goal에 도착
  * goal 성능에 도달하기까지 필요한 학습 데이터양이 많고, 시간도 많이 걸릴 수 있다. 
* 적절한 배치 크기는 상대적으로 less noisy하므로 적절하게 goal에 도착 
  * goal 성능 도달까지 필요한 학습 데이터양이 상대적으로 적을 수 있고, 시간도 보다 빠르게 도달할 수 있다. 
* 그러면 배치 사이즈가 크면 학습 효율이 선형적으로 좋다고 말할 수 있을까?
  * naive하게는 그렇다. 
* 하지만 너무 배치 사이즈가 크면... 한 스텝에 쓰는 양은 많아지는데 비해 방향의 정확성이 크게 개선되는 것은 없다. 
  * 즉 배치 사이즈의 증가가 학습 효율의 증가로 이어지지 않는다(saturated 예상)
 
![](./images/batch/01.png)

* 최적 step size for moment(learning rate)가 Batch가 크면 거의 선형적으로 증가하다 Saturated된다.
![](./images/batch/02.png)


## Gradient Noise Scale 에 대한 이론과 예측 

![](./images/batch/03.png)
![](./images/batch/04.png)
![](./images/batch/05.png)
![](./images/batch/06.png)
![](./images/batch/08.png)
![](./images/batch/07.png)

#### small batch growing인 경우, B << B_noise 인 경우  => delta_L 는 B에 비례 => 즉 배치가 커지면 성능 향상이 있다. 
#### large batch growing인 경우, B >> B_noise 인 경우  => delta_L 는 B에 무관 => 즉 배치가 더 커져도  성능 향상이 더이상 없다. 


## Simple Gradient Noise Scale 구하기 

* 이론적인 값을 구하려면 Hessian을 구해야 한다. 
* 비현실적인 가정, 즉 optimization이 well-condition되어 있다면(ex. convex), Hessian=Identity가 되므로..

![](./images/batch/09.png)

* noise scale은 각 요소별 gradient의 varience의 합을 글로벌 norm of gradient로 나눈 값
* 즉 varience에 대비해서 얼마나 gradient가 큰지를 나타내는 척도 
* 예측한 gradient가 true값에 비해서 L2 공간에서 얼마나 가까워졌는지를 나타내는 척도

![](./images/batch/10.png)

* simple B를 구하면 실제 B_noise을 대체적으로 맞게 구하더라 (differ only by a small constant multiplicative factor)
* 실용적으로는 simple B 공식을 써라 

![](./images/batch/11.png)

## Predictions for Data/Time Efficiency Tradeoffs

* 이때까지는 분석은 sinle point in loss landscape에서 국한되었지만..
* 이를 전체 학습(full training runs)에 확장시켜 보면 다음과 같은 시간-공간 trade-off 공식을 유도한다. (Appendix D)

![](./images/batch/12.png)

* S_min : 목표 성능에 도달하기까지 걸린 최소 스텝(시간) <- 다양한 배치 크기를 시도해봐서...
* S : 특정 실험에서의 목표 성능까지 도달한 시간
* E_min : 목표 성능에 도달하기까지 필요한 학습 인스턴스 개수(데이터량)  <- 다양한 배치 크기를 시도해봐서...
* E: 특정 실험에서의 목표 성능까지 필요했던 데이터량

### Critical B

* 학습 전반을 고찰해서 얻은 B값으로, 이론적으로는 B_critical = B_noise 일 것임

![](./images/batch/13.png)


### 데이터/시간 곡선

* 목표는 unreachable한 economically feasible에 들어가는 것
* 너무 작은 배치는 예산은 적게 쓰지만(적은 장비), 학습이 너무 오래 걸림 
* 너무 큰 배치는 학습은 빠르게 되지만, 예산이 많이 소요(많은 장비)
* 적절해야 economically feasible

![](./images/batch/14.png)

![](./images/batch/17.png)

## 우리 이론의 약점(caveats)

### short-horizon bias 
* locally하게 batch size adaptation한다는 가정
* greedy한 local improvement가 꼭 globally한 improvement로 이루어지지 않을 수도 

### poor-conditioning 

* poor-condition이라면(ex. 완전 찌부러진 타원), optimization이 osciliation 하게 된다. 
* 이 경우 우리의 방정식이 맞지 않는다. 

### simplified noise scale
* H=1이라는 극단적 가정

### Learning rate tuning: 
* 매 순간 learning rate를 opt값으로 찾는다는 비현실적 가정
* 똑똑한 learning rate scheduling을 쓰는 정도로 커버 될지도 

### Qudratic approx
* 테일러 급수 전개시 3차식 이상은 버림 

### Generalization 
* 학습 loss를 최적화할 때의 상황...
* 어떤 이는 말하기를 적절한 배치 크기가 학습 효율 측면에서는 좋지만(good train loss), test loss는 증가시킨다는 경험



## 확장성 있는 설명/예측들 

### Larger for difficult tasks

* 적절한 배치 크기는 테스크가 어려울수록 커질 것이다 (왜냐하면 데이터간 상관도가 낮아서 배치 내의 variance sum이 커지므로)
  * ex) B for MNIST < B for CIFAR-100
  * ex) 강화학습에서의 B가 상대적으로 크다. 
  * ex) Generative Model에서의 B값은 상대적으로 작다
  
![](./images/batch/15.png)
  
### Growth over training

* 학습이 되면 B값은 커진다. 
* 이는 학습이 충분히 될수록 수렴성이 강해저셔 |G| 값이 작아지므로

### Weak dependence on model size

* 동일 도메인이고 동일 모델이라면 모델 크기의 크고 작고랑은 큰 상관이 없더라 
* 큰 모델이 |G|를 작게 해서 상대적으로 큰 B를 나타낼 수도 있더라

![](./images/batch/16.png)

